# Práctica 1.A Toma de contacto con AIMA

##### Grupo 10: Belén Sánchez Centeno y Martín Fernández de Diego

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se **representan** algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  

Cuando termines los ejercicios entrega este archivo en el campus. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [1]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [2]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [3]:
p = Jarras()
p.initial

(0, 0)

In [4]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [5]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [6]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [7]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [8]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [9]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [10]:
misioneros.initial

(3, 3, 0)

In [11]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [12]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        return accs     

    ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES.
    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        else:
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        return tuple(l)
    
    def h(self, node):
        "Return the heuristic value for a given state."
        return 1

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [13]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [14]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [15]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [16]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: list index out of range

In [17]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [18]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [3]:
cd aima-python

[Errno 2] No such file or directory: 'aima-python'
/Users/martin/Documents/Cursos/2020-2021/IA/Practicas/Python/P1/aima-python


In [20]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [21]:
## resolvemos el problema de las jarras con el método de búsqueda en anchura.  

In [22]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [23]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8

In [24]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura breadth_first_tree_search(misioneros).solution()

In [25]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [26]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [27]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. ¿no debería terminar? escribe al final del ejercicio tus conclusiones.

KeyboardInterrupt: 

In [28]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [29]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [30]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [39]:
depth_first_graph_search(estado).solution()

['Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover h

In [31]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:

1. breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution():\
Como la búsqueda en anchura es completa, si la ejecución no termina, es porque esa configuración inicial no tiene solución.
2. depth_first_graph_search(estado).solution():\
Como la búsqueda en profundidad con repeticiones es completa, si la configuración inicial tiene solución la ejecución debe terminar. En este caso, la ejecución termina pero tarda mucho.

### Ejercicio 3:  Definición de heurísticas

#### Para el problema de los misioneros define una heurística y comenta sus propiedades
Heurística:\
Si no tenemos en cuenta la precondición de que los caníbales se comen a los misioneros.\
En esta interpretación, como hace falta una persona que lleve la barca, nos costará dos viajes transportar una persona a la otra orilla y volver a la posición previa. También debemos tener en cuenta que la barca tiene que quedar en la orilla derecha, un matiz que resolvemos restando el tercer atributo del estado.

In [32]:
# Heuristicas para el problema de los misioneros.

def misioneros_heuristica(node):
    state = node.state
    h = 2 * (state[0] + state[1]) - state[2]
    return h

#### Para el problema del puzle de 8 se pide definir al menos las siguientes funciones heurísticas:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [33]:
# Heuristicas para el 8 Puzzle.
# Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre de la función. 
import math

def linear(node):
    state = node.state
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    numDescolocadas = 0;
    for i in range(0,8):
        if node.state[i] != goal[i]:
            numDescolocadas += 1
    return numDescolocadas

def manhattan(node):
    state = node.state
    mhd = 0;
    for i in range(0,8):
        fila_state = i // 3
        col_state = i % 3
        
        piece = state[i]
        if piece == 0:
            fila_goal = 2
            col_goal = 2
        else:
            fila_goal = (piece - 1) // 3
            col_goal = (piece -1) % 3
        
        mhd += abs(fila_state - fila_goal) + abs(col_state - col_goal)
    return mhd

def sqrt_manhattan(node):
    state = node.state
    mhd = 0;
    for i in range(0,8):
        fila_state = i // 3
        col_state = i % 3
        
        piece = state[i]
        if piece == 0:
            fila_goal = 2
            col_goal = 2
        else:
            fila_goal = (piece - 1) // 3
            col_goal = (piece -1) % 3
        
        mhd += abs(fila_state - fila_goal) + abs(col_state - col_goal)
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente **estado inicial** y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [34]:
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [35]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [36]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [37]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [38]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [39]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo. Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo
<br>

In [40]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [41]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

2.91 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

2.72 ms ± 92.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

33.1 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

3.06 ms ± 281 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.
Hemos analizado las heurísticas: linear, manhattan, sqrt_manhattan y max_heuristic.\
Visto el cálculo en tiempo del algoritmo A estrella en las distintas heurísticas, podemos concluir:
**La mejor heurística es la Manhattan.**\
Atendiendo a sus características, podemos decir que la manhattan es la mejor porque es la mas informada. Veámoslo:
* manhattan es más informada que linear (manhattan(n) >= linear(n)) porque manhattan(n) $\in$ (1,...,4) y linear(n) = 1 si la pieza está mal colocada, en cambio ambas son 0 si la pieza está correcta.
* manhattan es más informada que sqrt_manhattan porque manhattan(n) >= sqrt(manhattan(n)) = sqrt_manhattan(n) para cualquier n.
* max_heuristic tiene que calcular las heurísticas linear y manhattan y después elegir la mejor. Esto ralentiza el cálculo y por tanto es peor que manhattan.

## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [45]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0        

In [46]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p = Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [47]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [48]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [49]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [50]:
astar_search(p8,manhattan).solution()

['Mover hueco derecha']

In [51]:
astar_search(p8p,manhattan).solution()

['Mover hueco derecha']

In [52]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [53]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


### Ejercicio 5:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

In [54]:
E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
p1 = Ocho_Puzzle(E1)
p2 = Ocho_Puzzle(E2)
p3 = Ocho_Puzzle(E3)
p4 = Ocho_Puzzle(E4)

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 1 | 3 |   | 1 | 0 | 3 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 4 | 8 | 6 |   | 4 | 8 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 2 | 5 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**. 

                                       E1           E2           E3          E4
                                       
    Anchura                            L= 17        L= 11        L= 20       L= 3
                                       T= 6.81s     T= 55.8 ms   T= 1min 23s T= 39.4 µs 
                                       NA= 13634    NA= 1255     NA= 57057   NA= 13
   
    Profundidad                        L= 7977      L= 52429     L= 39720    L= 27  
                                       T= 7.83 s    T= 6min 25s  T= 3min 11s T= 283 µs
                                       NA= 8198     NA= 60036    NA= 42926   NA= 28

                                       
    Coste Uniforme                     L= 17        L= 11        L= 20       L= 3
                                       T= 45 s      T= 167 ms    T= 9m 45s   T= 153 µs 
                                       NA= 14092    NA= 870      NA= 48428   NA= 13
                                                                            
    Primero el mejor (linear)          L= 39        L= 33        L= 76       L= 3
                                       T= 8.29 ms   T= 2.34 ms   T= 129 ms   T= 40.1 µs 
                                       NA= 171      NA= 78       NA= 747     NA= 4
                                                                            
    Primero el mejor (manhattan)       L= 25        L= 15        L= 38       L= 3 
                                       T= 8.97 ms   T= 1.47 ms   T= 13.5 ms  T= 48.3 µs 
                                       NA= 175      NA= 53       NA= 220     NA= 4
                                                                             
    A* (linear)                        L= 17        L= 11        L= 20       L= 3 
                                       T= 162 ms    T= 2.54 ms   T= 2.31 s   T= 45.3 µs 
                                       NA= 433      NA= 74       NA= 3230    NA= 4
                                                                             
    A* (manhattan)                     L= 17        L= 11        L= 20       L= 3
                                       T= 51 ms     T= 1.42 ms   T= 640 ms   T= 50.7 µs
                                       NA= 433      NA= 49       NA= 1636    NA= 4

Vistos los resultados previos, podemos concluir que las búsquedas ciegas son mucho menos eficientes que las búsquedas informadas. Es decir, a medida que el número de nodos aumenta, la búsqueda ciega se convierte en un método poco útil; mientras que la búsqueda informada, obtiene unos resultados muy satisfactorios.\
Además, dentro de la búsqueda informada, podemos apreciar diferencias entre el número de nodos generados por las distintas heurísticas: por norma general, la heurística manhattan generó menos nodos que la heurística linear. Esto provoca que el tiempo de ejecución se reduzca aproximadamente de forma proporcional.


### Ejercicio 6:  En el ejercicio anterior hemos hecho pruebas con distintos estados iniciales. Explica cómo afecta a la resolución del problema si lo que cambiamos es el estado objetivo.
En vez de  goal = (1, 2, 3, 4, 5, 6, 7, 8, 0) queremos usar goal2 = (1, 2, 3, 4, 0, 5, 6, 7, 8).
####  No es necesario repetir los experimentos pero sí justificar convenientemente la respuesta dada.

Dado otro objetivo, si la búsqueda es informada, el resultado es parecido al del ejercicio anterior o proporcional entre los distintos estados iniciales, algoritmos de búsqueda y heurísticas, porque le estoy dando información para que lo resuelva igual que lo resolvía antes; pero, si la búsqueda es ciega, puede que este objetivo sea más fácil de alcanzar que antes y que disminuyan los tiempos; aunque las conclusiones sobre eficiencia que sacamos son las mismas.

### Ejercicio 7: (opcional) Definir nuevas heurísticas (más informadas) y completa una nueva fila de la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [55]:
# Proponemos como ejemplo la heurística h3 que se obtiene sumando a la heurística
# manhattan una componente que cuantifica la "secuencialidad" en las
# casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj
# ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en
# A*, con cada uno de los estados anteriores. Comentar los resultados.

In [56]:
def h3(node):
    # En cada casilla almacenamos el indice de la posicion siguiente en sentido horario.
    next_ocho_puzle = (1,2,5,0,4,8,3,6,7)
      
    def sec(node):
        state = node.state
        sol_aux = 0
        sol = 0
        
        for i in range(0,8):
            piece = state[i]
            
            if piece == 0: sol_aux = 0
            elif i == 4: sol_aux = 0
            else:
                if piece < 8: piece_next = piece + 1
                else: piece_next = 1
                if piece_next == state[next_ocho_puzle[i]]: sol_aux = 1
                else: sol_aux = 2
                
            sol += sol_aux
        
        return sol
            
    return manhattan(node) + sec(node)

In [57]:
resuelve_ocho_puzzle(E1, astar_search, h3)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 17. Nodos analizados: 405


In [58]:
%%timeit
astar_search(p1,h3)

47.3 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
resuelve_ocho_puzzle(E2, astar_search, h3)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 11. Nodos analizados: 47


In [60]:
%%timeit
astar_search(p2,h3)

1.58 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [61]:
resuelve_ocho_puzzle(E3, astar_search, h3)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 20. Nodos analizados: 1378


In [62]:
%%timeit
astar_search(p3,h3)

585 ms ± 40.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
resuelve_ocho_puzzle(E4, astar_search, h3)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 3. Nodos analizados: 4


In [64]:
%%timeit
astar_search(p4,h3)

86.8 µs ± 7.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


                                       E1           E2           E3          E4
                                       
    A* (h3)                            L= 17        L= 11        L= 20       L= 3
                                       T= 47.3 ms   T= 1.58 ms   T= 585 ms   T= 86.8 µs
                                       NA= 405      NA= 47       NA= 1378    NA= 4

##### Conclusiones
A la vista de los resultados de la heurística h3, hemos conseguido reducir el número de nodos analizados en todos los estados iniciales.\
En cambio, el tiempo de cómputo solo ha mejorado en los casos E1 y E3, donde se ha reducido más significativamente el número de nodos. Y es que el incremento de la complejidad computacional, hace que aumente también el tiempo por ciclo.

### Ejercicio 8. Puzle de 15.
En 1878, Sam Loyd daba un premio de 1.000 dolares a quien fuera capaz de resolver su famoso 15 Puzzle (n=16). Se trataba de un puzzle deslizante de 16 piezas que *no tenía solución* debido que se necesitaba un número impar de movimientos y, como hemos visto, sólo un número de movimientos par tiene solución. Muchas personas trataron de resolverlo.\
El número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución.\
En el caso del siguiente estado inicial que sí tiene solución:  (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


     +---+---+---+---+  
     | 1 | 2 | 3 | 4 |
     +---+---+---+---+
     | 5 | 6 | 7 | 8 |
     +---+---+---+---+
     | 9 |10 |12 |15 |
     +---+---+---+---+
     |13 |14 |11 | 0 |
     +---+---+---+---+
     
     Analicemos el árbol de posibilidades:
     (Véase que basta con representar la submatriz 2x2 de la esquina inferior derecha)
     
     +---+---+     +---+---+
     |12 |15 |     |12 |15 |
     +---+---+ --> +---+---+
     |11 | 0 |     | 0 |11 |
     +---+---+     +---+---+
                   +---+---+     +---+---+     +---+---+     +---+---+
                   |12 | 0 |     | 0 |12 |     |11 |12 |     |11 |12 |
               --> +---+---+ --> +---+---+ --> +---+---+ --> +---+---+ - Objetivo
                   |11 |15 |     |11 |15 |     | 0 |15 |     |15 | 0 |
                   +---+---+     +---+---+     +---+---+     +---+---+
                   
     5 nodos generados.
     
     
     
     Si tuviéramos un problema análogo en un puzle de 8,
     
     +---+---+---+ 
     | 1 | 2 | 3 |
     +---+---+---+
     | 4 | 6 | 8 |
     +---+---+---+
     | 7 | 5 | 0 |
     +---+---+---+
     
     Con A* también obtendríamos un árbol de exploración análogo:
     
     +---+---+     +---+---+
     | 6 | 8 |     | 6 | 8 |
     +---+---+ --> +---+---+
     | 5 | 0 |     | 0 | 5 |
     +---+---+     +---+---+
                   +---+---+     +---+---+     +---+---+     +---+---+
                   | 6 | 0 |     | 0 | 6 |     | 5 | 6 |     | 5 | 6 |
               --> +---+---+ --> +---+---+ --> +---+---+ --> +---+---+ - Objetivo
                   | 5 | 8 |     | 5 | 8 |     | 0 | 8 |     | 8 | 0 |
                   +---+---+     +---+---+     +---+---+     +---+---+
                   
      5 nodos generados.